In [ ]:
import os

start = 1
stop = 50

for i in range(start,stop):
    #job_file = open(f"/home/vadym.voitsekhovskyi/lstchain_work/jobs/r0_to_dl_mc_run{i}.job", "w")
    job_file = open(f"/Users/vdk/files/job{i}.txt", "w")

    job_file.writelines("#!/bin/bash")
    job_file.writelines("#BATCH --partition=long\n")
    job_file.writelines("#SBATCH --time=1:59:00\n")
    job_file.writelines("#SBATCH --ntasks=1\n")
    job_file.writelines("#SBATCH --mem-per-cpu=5000 # in MB\n")
    job_file.writelines("#SBATCH --output=/fefs/aswg/workspace/vadym.voitsekhovskyi///lstchain_slurm/log/job_sim_telarray_parameter_scan_1.log\n")
    job_file.writelines("#SBATCH --error=/fefs/aswg/workspace/vadym.voitsekhovskyi///lstchain_slurm/error/job_sim_telarray_parameter_scan_1.err\n")
    job_file.writelines("\n")
    job_file.writelines(f"srun /home/vadym.voitsekhovskyi/mambaforge/envs/ctapipe/lib/python3.9/site-packages/lstchain/scripts/lstchain_mc_r0_to_dl1.py --input-file /fefs/aswg/data/mc/DL0/LSTProd2/TrainingDataset/Protons/dec_2276/sim_telarray/node_theta_16.087_az_108.090_/output/corsika_theta_16.087_az_108.090_run{i}.simtel.gz --output-dir ~/lstchain_output\n")
    job_file.close()

for i in range(start,stop):
    command = f"sbatch r0_to_dl_mc_run{i}.job"
    os.system(command)
    print(f"Sbatch number {i} runned")


## Build histogram for energies (mono/stereo variant)

In [ ]:
from ctapipe.io import EventSource

energies_all = []
for k in range(1,41):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    for event in event_iterator:
        #if event.trigger.tels_with_trigger > 1:
        energies_all.append(j.simulation.shower.energy.to_value())

histplot(energies_all, log_scale = True, bins = 45)
plt.xlabel("Energy in TeV")
plt.title("Shower energies for LST")
#plt.savefig("/Users/vdk/Shower_energies.png")

## Process and extract optical efficiency for stereo trigger events

In [ ]:
from ctapipe.io import EventSource
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.calib import CameraCalibrator
import numpy as np
import time


int_cut = 0.4
min_pixel = 80
event_container = {}
t1 = time.perf_counter(), time.process_time()


muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})


muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})


for k in range(1,41):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run3983.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    keks = k*1000 # variable that denotes simulation run to use as key in the dictionary
    for i,event in enumerate(event_iterator):
        if len(j.trigger.tels_with_trigger) > 1:            
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            calib(event)
            image_processor(event)
            muon_processor(event)
            for tels in event.muon.tel.keys():
                if not np.isnan(event.muon.tel[tels].efficiency.width):
                    event_container[keks+i] = event
        print("iteration succesful")
    t2 = time.perf_counter(), time.process_time()
        

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

            
        

## Time cleaning of stereo trigger events

In [ ]:
from ctapipe.io import EventSource
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.calib import CameraCalibrator
import numpy as np
import time

int_cut = 0.2
min_pixel = 150
checker = 0
cleaned_event_container = {}
t1 = time.perf_counter(), time.process_time()


muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})


muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

for k in range(1,41):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run3983.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,event in enumerate(event_iterator):
        checker += 1
        if len(event.trigger.tels_with_trigger) > 1: 

            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)

            for tels in event.trigger.tels_with_trigger:
                pixel_mask = []
                for k,l in enumerate(event.r0.tel[tels].waveform[1]):
                    if np.argmax(l) in range(16,40): 
                        pixel_mask.append(True)
                    else:
                        pixel_mask.append(False)
                working_wave = event.r1.tel[tels].waveform.copy()
                for k,l in enumerate(working_wave):
                    if pixel_mask[k]:
                        # two variants for non-muons mask
                        #working_wave[k] = np.zeros(40)     
                        working_wave[k] = np.mean(event.r1.tel[tels].waveform, axis = 0)
                event.r1.tel[tels].waveform = working_wave
            
                calib(event)
                image_processor(event)
                muon_processor(event)

        for m in event.muon.tel.keys():
            if not np.isnan(event.muon.tel[m].efficiency.width):
                cleaned_event_container[keks+i] = event
        t2 = time.perf_counter(), time.process_time()
        




            
            
print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")      


### Extracting muon rings from cleaned events

In [ ]:
# PROCESSING OF CLEANED IMAGES
lets_check = 0
opt_eff_cleaned = []
events_opt_eff_cleaned = []

r1_waves = []
n_pixel = []
energies = []
int_ratio = []
clean_event_number = []

for i in cleaned_event_container:
    for m in cleaned_event_container[i].trigger.tel.keys():
        if not np.isnan(cleaned_event_container[i].muon.tel[m].efficiency.width):
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                
                #print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                if cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.7 and cleaned_event_container[i].muon.tel[m].parameters.completeness > 0.5 and cleaned_event_container[i].muon.tel[m].parameters.mean_squared_error.to_value() < 0.01:
                #if cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 30:
                    #if abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.2:
                    #print("n_pixels ", cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                    #print("r1 waveform", len(np.where(cleaned_event_container[i].r1.tel[m].waveform > 2)[0]))
                    r1_waves.append(len(np.where(cleaned_event_container[i].r1.tel[m].waveform > 2)[0]))
                    n_pixel.append(cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                    energies.append(cleaned_event_container[i].simulation.shower.energy)
                    int_ratio.append(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio)
                    #print("len from r1 ", len((np.where(cleaned_event_container[i].r1.tel[m].waveform > 2)[0])))
                    #print("MSE ", cleaned_event_container[i].muon.tel[m].parameters.mean_squared_error)                
                    #print("Muon parameters ", cleaned_event_container[i].muon.tel[m].parameters)   
                    #print("intensity_check ", abs(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                    #print("Muon parameters ", cleaned_event_container[i].muon.tel[m].parameters)   
                    #print("intensity_check ", abs(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                    lets_check+=1
                    plt.figure(figsize = (9,6))
                    camgeom = source.subarray.tel[m].camera.geometry
                    #title=f"event_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container[i].trigger.tel.keys())}"
                    title=f"CleanedRing_{i}_optEff{round(cleaned_event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(cleaned_event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggeredTel{m}"
                    #title=f"TrueImage for events tagged as muon ring by cleaning algorithm {i}"
                    disp = CameraDisplay(camgeom,title=title)
                    disp.image = cleaned_event_container[i].simulation.tel[m].true_image
                    #disp.image = cleaned_event_container[i].r1.tel[m].waveform.sum(axis=1)
                    #disp.image = cleaned_event_container[i].simulation.tel[m].true_image
                    disp.cmap = plt.cm.RdBu_r
                    disp.add_colorbar()
                    disp.set_limits_percent(95)
                    clean_event_number.append(i)
                    plt.show()
                    #plt.savefig(f"/Users/vdk/ImagePurgatorium/TrueImagesOfCleanedMuons_intensitycut<0.3_event{i}.png")
                    opt_eff_cleaned.append(cleaned_event_container[i].muon.tel[m].efficiency.optical_efficiency)
                    events_opt_eff_cleaned.append(i)
                    
plt.close()

### Procesing of regular image

In [ ]:
# PROCESSING OF REGULAR IMAGES
lets_check = 0
opt_eff_cleaned = []
reg_event_number = []
for i in event_container:
    for m in event_container[i].trigger.tel.keys():
        if not np.isnan(event_container[i].muon.tel[m].efficiency.width):
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                
                print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                #print("MSE ", event_container[i].muon.tel[m].parameters.mean_squared_error)                
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                #if event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.1:
                #if abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.2:
                #print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                lets_check+=1
                plt.figure(figsize = (9,6))
                camgeom = source.subarray.tel[m].camera.geometry
                title=f"Event_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container[i].trigger.tel.keys())}"
                #title=f"CleanedRing_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggeredTel{m}"
                disp = CameraDisplay(camgeom,title=title)
                disp.image = event_container[i].simulation.tel[m].true_image
                #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
                disp.cmap = plt.cm.RdBu_r
                disp.add_colorbar()
                disp.set_limits_percent(95)
                reg_event_number.append(i)
                plt.show()
                #plt.savefig(f"/Users/vdk/ImagePurgatorium/MuonsWithoutCleaning_intensitycut<0.3_event{i}.png")
                #opt_eff_cleaned.append(event_container[i].muon.tel[m].efficiency.optical_efficiency)
                    
plt.close()|

## Calculate triggered telescopes

In [ ]:
###
# Calculate triggered telescopes
###
int_cut = 0.4
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
event_container = {}
t1 = time.perf_counter(), time.process_time()

tels_1_2 = []
tels_1_3 = []
tels_1_2_3 = []
tels = []

for k in range(1,41):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        checker += 1
        if len(j.trigger.tels_with_trigger) > 1:            
            tels.append(j.trigger.tels_with_trigger)
    t2 = time.perf_counter(), time.process_time()
        

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
#
#
# Need this to fill optical efficiency array with new values
#
#
optical_coef = []
event_optical_coef = []
for i in event_container:
    for m in event_container[i].trigger.tel.keys():
        #print(np.isnan(event_container[i].muon.tel[m].efficiency.optical_efficiency))
        #if cleaned_event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(cleaned_event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.3 and cleaned_event_container[i].muon.tel[m].parameters.completeness > 0.5:
        if not np.isnan(event_container[i].muon.tel[m].efficiency.optical_efficiency) and event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50:
            optical_coef.append(event_container[i].muon.tel[m].efficiency.optical_efficiency)
            event_optical_coef.append(i)

print(np.mean(optical_coef))
print(np.std(optical_coef))
print(np.std(optical_coef)/np.mean(optical_coef))
print(len(optical_coef))
#optical_coef = optical_coef[0:31]

In [ ]:
#
#
# Code for extracting events which are presented in both cleaned and raw dataset
#
#

# dictionary with event:opticalefficiency for original events
res_dict = {event_optical_coef[i]:optical_coef[i] for i in range(len(optical_coef))}
# dictionary with event:opticalefficiency for cleaned events
res_dict_cleaned = {events_opt_eff_cleaned[i]:opt_eff_cleaned[i] for i in range(len(opt_eff_cleaned))}
print(len(res_dict))
print(len(res_dict_cleaned))

plot = []
for i in res_dict_cleaned.keys():
    if i in res_dict.keys():
        plot.append(res_dict[i])
    else:
        plot.append(0)

new_plot = []
new_ind = []
for i,j in enumerate(plot):
    if not j == 0:
        new_plot.append(j)
        new_ind.append(i)


        
print("len of remained = ", len(new_plot))
print("len of all = ", len(opt_eff_cleaned))

cumulative_dif = 0

for i in res_dict:
    if i in res_dict_cleaned:
        cumulative_dif += abs(res_dict[i] - res_dict_cleaned[i])

print("cumulative_dif = ", cumulative_dif)

print("relative cumulative_dif = ", cumulative_dif/len(new_plot))

In [ ]:
#
#
# Images for events without cleaning
#
#




x_range_1 = [i for i in range(len(optical_coef))]
#std_arr = [np.std(optical_coef)/np.mean(optical_coef) for i,j in enumerate(optical_coef)]
std_arr_1 = [np.std(optical_coef)/i**0.5 for i,j in enumerate(optical_coef)]
std_arr_1_big = [np.std(optical_coef) for i,j in enumerate(optical_coef)]
#std_arr = [np.std(opt_eff) for i,j in enumerate(opt_eff)]


mean_range_1 = [i-1 for i in range(len(optical_coef)+3)]
y_upper_1 = [np.mean(optical_coef)+np.std(optical_coef) for i in mean_range_1]
y_lower_1 = [np.mean(optical_coef)-np.std(optical_coef) for i in mean_range_1]
y_mid_1 = [np.mean(optical_coef) for i in mean_range_1]
y_mid_1 = [np.mean(optical_coef[:i+1]) for i in range(len(optical_coef[:i+1]))]


#y_upper_1 = [j+std_arr_1[-1] for i,j in enumerate(y_mid_1)]
#y_lower_1 = [j-std_arr_1[-1] for i,j in enumerate(y_mid_1)]

y_upper_1 = [j+std_arr_1[-1] for i,j in enumerate(y_mid_1)]
y_lower_1 = [j-std_arr_1[-1] for i,j in enumerate(y_mid_1)]


#y_upper_1 = [j+np.std(optical_coef[:i+1]) for i,j in enumerate(y_mid_1)]
#y_lower_1 = [j-np.std(optical_coef[:i+1]) for i,j in enumerate(y_mid_1)]


y_upper_1 = [j+std_arr_1[-1] for i,j in enumerate([np.mean(optical_coef) for i in x_range_1])]
y_lower_1 = [j-std_arr_1[-1] for i,j in enumerate([np.mean(optical_coef) for i in x_range_1])]

plt.figure(figsize = (12,6))
plt.ylim(0.095,0.34)

plt.style.use('default')
plt.xlim(-1,len(optical_coef))
plt.xlabel("Event")
#plt.ylabel("Optical eff with rel. error for 1 measurment")
plt.ylabel("Optical efficiency with std")
#plt.title("Opt eff for muon rings without performing additional cleaning int_cut<0.1")
plt.title("Events before additional cleaning (pix > 50, intensity_cut = 0.3, completeness>0.5)")
plt.grid(alpha=0.5)
#plt.ylim(0,0.4)

plt.scatter(x_range_1, optical_coef)
plt.errorbar(x_range_1, optical_coef, yerr=std_arr_1_big, fmt='o', ms = 7, alpha = 0.8, color = 'darkorange')#, c = 'blue')#, s = 100)#, c= 'black')


#plt.fill_between(x_range_1, y_upper_1, y_lower_1, color = 'green', alpha = 0.3, label = 'mean +/- $1\sigma$')
plt.fill_between(x_range_1, y_upper_1, y_lower_1, color = 'green', alpha = 0.3, label = 'mean +/- $1\sigma$')
#plt.plot(x_range_1, y_mid_1, label = "mean", c = 'green', linewidth = 1.5)
plt.plot(x_range_1, [np.mean(optical_coef) for i in x_range_1], label = "mean", c = 'green', linewidth = 1.5)


plt.legend()
print("mean = ", np.mean(optical_coef))
print("std = ", np.std(optical_coef))
print("rel_err = ", np.std(optical_coef)/np.mean(optical_coef))
print("len = ", len(optical_coef))
#plt.savefig("/Users/vdk/ImagePurgatorium/LSTprotonsWithoutCleaningIntCut30%_Containment>50%_v2.png")

In [ ]:
#
#
# Images for events after cleaning
#
#

#plt.style.use('tableau-colorblind10')
plt.style.use('default')


#std_arr_3 = []

#for i,j in enumerate(optical_coef):
#    std_arr_3.append(np.std(optical_coef[:i+1])/(i**0.5))

#print(len(new_ind), len(std_arr_3))   

#std_arr = []

#for i,j in enumerate(opt_eff_cleaned):
#    std_arr.append(np.std(opt_eff_cleaned[:i+1])/(i**0.5))
    #print(np.std(opt_eff_cleaned[:i+2]))
    #print(len(opt_eff_cleaned[:i+1]))


mean_range = [i-1 for i in range(len(opt_eff_cleaned)+2)]
mean_range = [i-1 for i in range(len(opt_eff_cleaned)+2)]

y_mid = [np.mean(opt_eff_cleaned) for i in mean_range]
y_mid = [np.mean(opt_eff_cleaned[:i+1]) for i in range(len(opt_eff_cleaned[:i+1]))]


y_upper = [np.mean(opt_eff_cleaned)+np.std(opt_eff_cleaned) for i in mean_range]
y_lower = [np.mean(opt_eff_cleaned)-np.std(opt_eff_cleaned) for i in mean_range]



x_range = [i for i in range(len(opt_eff_cleaned))]




#std_arr = [np.std(optical_coef)/np.mean(optical_coef) for i,j in enumerate(optical_coef)]
std_arr = [np.std(opt_eff_cleaned)/i**0.5 for i,j in enumerate(opt_eff_cleaned)]
std_arr_big = [np.std(opt_eff_cleaned) for i,j in enumerate(opt_eff_cleaned)]
std_arr_3 = [np.std(new_plot) for i,j in enumerate(new_plot)]
#std_arr = [np.std(opt_eff) for i,j in enumerate(opt_eff)]


plt.figure(figsize = (12,6))
plt.ylim(0.095,0.34)
#plt.ylim(0.095,0.265)
plt.xlabel("Event")
#plt.ylabel("Optical eff with rel. error for 1 measurment")
plt.ylabel("Optical efficiency value")
#plt.title("Opt eff for muon rings after time sample cleaning, pix > 50, intensity<0.2")
plt.title("Events after additional cleaning (pix > 50, intensity_cut = 0.3, time_bin<16, completeness>0.5)")
#plt.title("Events after island cleaning (size $\leq 4$)")


plt.grid(alpha=0.5)
#plt.ylim(0.05,0.3)
#plt.scatter(new_ind, new_plot, color = 'orange')


y_upper = [j+std_arr[-1] for i,j in enumerate(y_mid)]
y_lower = [j-std_arr[-1] for i,j in enumerate(y_mid)]


y_upper = [j+std_arr[-1] for i,j in enumerate([np.mean(opt_eff_cleaned) for i in x_range])]
y_lower = [j-std_arr[-1] for i,j in enumerate([np.mean(opt_eff_cleaned) for i in x_range])]

plt.errorbar(x_range, opt_eff_cleaned, yerr=std_arr_big, fmt='o', ms = 5, alpha = 0.8, label="cleaned")#, c = 'blue')#, s = 100)#, c= 'black')
plt.errorbar(new_ind, new_plot, yerr=std_arr_3, fmt='o', ms = 5, alpha = 0.8, label="original") #, c = 'blue')#, s = 100)#, c= 'black')
#plt.scatter(new_ind, new_plot, alpha = 0.9, color = "orange", label="original") #, c = 'blue')#, s = 100)#, c= 'black')


#plt.fill_between(mean_range, y_upper, y_lower, alpha = 0.3, color = 'green', label = 'mean +/- $1\sigma$')
plt.fill_between(x_range, y_upper, y_lower, alpha = 0.3, color = 'green', label = 'mean +/- $1\sigma$')
#plt.plot(mean_range, y_mid, label = "mean", lw = 1.5, color='green')
#plt.plot(x_range, y_mid, label = "mean", lw = 1.5, color='green')
plt.plot(x_range, [np.mean(opt_eff_cleaned) for i in x_range], label = "mean", lw = 1.5, color='green')

#plt.fill_between(x_range_1, y_upper_1, y_lower_1, color = 'r', alpha = 0.3, label = 'mean +/- $1\sigma$')

plt.xlim(-1,len(opt_eff_cleaned))
plt.legend()
print("mean = ", np.mean(opt_eff_cleaned))
print("std = ", np.std(opt_eff_cleaned))
print("rel_err = ", np.std(opt_eff_cleaned)/np.mean(opt_eff_cleaned))
print("len of all = ", len(opt_eff_cleaned))
print("len of remained = ", len(new_plot))
#plt.savefig("/Users/vdk/ImagePurgatorium/CleanedOptEff_pix>50_int=01_16timebin_complet>05_RelStdErr.png")
#plt.savefig("/Users/vdk/ImagePurgatorium/IslandsCleaned_maskMEAN_Island4Pixel.png", dpi=900)

## Energy weighting of protons and muons for 1 LST configuration

In [ ]:
import numpy as np

filename1 = '/Users/vdk/energies_500.txt'
fi1 = open(filename1,'r')
lines = fi1.readlines() 
splitted = lines[0].split(',')
energies_proton = np.array(list(filter(None, splitted))).astype('float')
fi1.close()

filename2 = '/Users/vdk/energies_for_filerange1-499.txt'
fi2 = open(filename2, 'r')
lines = fi2.readlines()
energies_muon = np.array(lines).astype('float')
fi2.close()

w_protons = sum([energy**(-0.7) for energy in energies_proton])
#new_proton_sample = len(energies_proton)/w_protons * len(energies_proton)
new_proton_sample = w_protons
print("Initial proton sample size = ", len(energies_proton))
print("Weighted proton sample size = ", new_proton_sample)


w_muons = sum([energy**(-0.7) for energy in energies_muon])
#new_muon_sample = len(energies_muon)/w_muons * len(energies_muon)
new_muon_sample = w_muons
print("Initial muon sample size = ", len(energies_muon))
print("Weighted muon sample size = ", new_muon_sample)

print("Initial relation between muon sample size and proton sample = ", len(energies_muon)/len(energies_proton))
print("Weighed relation between muon sample size and proton sample = ", new_muon_sample / new_proton_sample)

## Muon Flagging

This makes Muon Flagging based on:
-   Number of pixels with R1 max waveform amplitude cut
-   Time samples
-   Mean of the variance for whole event in the considered time samples

In [ ]:
from ctapipe.io import EventSource
import numpy as np

max_timesample = 14
min_timesample = 10
r1_cut = 2
min_pixel_cut = 40
max_pixel_cut = 200
event_container_flag = {}

low_threshold_variance = 0.1 # choose events with variance higher than this value
up_threshold_variance = 0.4

for run in range(78,100):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run39{run}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator):
        if len(event.trigger.tels_with_trigger) > 1:        
            for tel_id in event_container[key].trigger.tels_with_trigger:
                tmp_arr = []
                var_arr = []
                for pix_id in event.r1.tel[tel_id].waveform[:,min_timesample:max_timesample]:
                    tmp_arr.append(np.max(pix_id)) # знаходить максимальне значення waveform в кожному пікселі
                    var_arr.append(np.var(pix_id))
                
                tmp_arr = np.array(tmp_arr)
                pixel_number = len(tmp_arr[tmp_arr>r1_cut])
                
                if (pixel_number > min_pixel_cut) and (pixel_number < max_pixel_cut) and (np.mean(var_arr) > low_threshold_variance) and (np.mean(var_arr) < up_threshold_variance):
                    event_container_flag[run*1000+i] = event



## Full Version of cleaning for monoscopic case

In [ ]:




max_time = 14

image_processor = ImageProcessor(source.subarray)

muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)



pixel_mask = []
for_clean_mask = []
tels = 1
original_r1_waveform = event.r1.tel[tels].waveform.copy()
for k,l in enumerate(event.r0.tel[tels].waveform[1]):
    if np.argmax(l) in range(17,41): 
        pixel_mask.append(True)
    else:
        pixel_mask.append(False)

working_wave = event.r1.tel[tels].waveform.copy()
for k,l in enumerate(working_wave):
    if pixel_mask[k]:
        for_clean_mask.append(True)
        #working_wave[k] = np.zeros(40)     
        working_wave[k] = np.mean(event.r1.tel[tels].waveform, axis = 0)
    else:
        for_clean_mask.append(False)
event.r1.tel[tels].waveform = working_wave
cam_geometry = source.subarray.tel[tels].camera.geometry
return_pixels = np.array([])
for_clean_mask = np.array(for_clean_mask)
n_islands, island_id = number_of_islands(cam_geometry, for_clean_mask)
for t in range(1,n_islands+1):
    if len(np.where(island_id == t)[0]) <= 3:
        return_pixels = np.append(return_pixels,np.where(island_id ==t)[0])
return_pixels = return_pixels.astype(int)
event.r1.tel[tels].waveform[return_pixels,:] = original_r1_waveform[return_pixels,:]  

calib(event)






In [ ]:
329*3.38

In [ ]:
322630*2.68